In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

#from tensorflow.data.Dataset import load
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import keras_tuner as kt
import pandas as pd

import os
from file_helpers import unpickle_from_file

2024-03-22 00:15:44.405437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_labels_sorted = unpickle_from_file('../data/processed_data/train_labels_sorted.bin')

In [3]:
#Read training data
batch_size = 75
img_height = 227
img_width = 227

train_ds = tf.keras.utils.image_dataset_from_directory(
  '../data/processed_data/training_images',
  validation_split=0.15,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
labels = train_labels_sorted)

Found 51957 files belonging to 8 classes.
Using 44164 files for training.


2024-03-22 00:15:49.116701: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
#Read validation data

val_ds = tf.keras.utils.image_dataset_from_directory(
  '../data/processed_data/training_images',
  validation_split=0.15,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
labels = train_labels_sorted)

Found 51957 files belonging to 8 classes.
Using 7793 files for validation.


In [5]:
#Try to keep things in memory for performance reasons
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [6]:
def model_builder(hp):
    img_height = 227
    img_width = 227

    num_classes = 8

    hp_depth_of_network = hp.Choice('depth_of_network', values=[3, 4, 5, 6])

    model_layers = [
      layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.2),
      layers.Flatten()
    ]
    
    dense_layers = []
    for i in range(0, (3 + hp_depth_of_network)):
        dense_layers.append(layers.Dense(256, activation='relu'))

    model_layers = model_layers + dense_layers

    model_layers.append(layers.Dense(num_classes))
    
    model = Sequential(model_layers)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=2,
                     factor=3,
                     directory='my_dir',
                     overwrite = True,#Don't use results cached in my_dir
                     project_name='intro_to_kt')

In [8]:
#We had some models with loss less than 1
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 1)

In [9]:
#Original example calls for 50, but training time will be
#very long for our model with 50. We'll do 10 max instead
tuner.search(train_ds, validation_data = val_ds, epochs=10, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 2 Complete [01h 14m 40s]
val_accuracy: 0.8181701302528381

Best val_accuracy So Far: 0.8181701302528381
Total elapsed time: 02h 28m 33s


In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 10 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_ds, validation_data = val_ds, epochs=10)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/10
589/589 [==============================] - 2288s 4s/step - loss: 3.8284 - accuracy: 0.8119 - val_loss: 0.7875 - val_accuracy: 0.8182
Epoch 2/10
589/589 [==============================] - 2290s 4s/step - loss: 0.7981 - accuracy: 0.8133 - val_loss: 0.7845 - val_accuracy: 0.8182
Epoch 3/10
589/589 [==============================] - 2263s 4s/step - loss: 0.7972 - accuracy: 0.8133 - val_loss: 0.7864 - val_accuracy: 0.8182
Epoch 4/10
589/589 [==============================] - 2292s 4s/step - loss: 0.7971 - accuracy: 0.8133 - val_loss: 0.7841 - val_accuracy: 0.8182
Epoch 5/10
589/589 [==============================] - 2274s 4s/step - loss: 0.7967 - accuracy: 0.8133 - val_loss: 0.7855 - val_accuracy: 0.8182
Epoch 6/10
589/589 [==============================] - 2274s 4s/step - loss: 0.7966 - accuracy: 0.8133 - val_loss: 0.7841 - val_accuracy: 0.8182
Epoch 7/10
589/589 [==============================] - 2314s 4s/step - loss: 0.7969 - accuracy: 0.8133 - val_loss: 0.7836 - val_accuracy:

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
history = hypermodel.fit(train_ds, validation_data = val_ds, epochs = best_epoch)

589/589 [==============================] - 2325s 4s/step - loss: 0.9995 - accuracy: 0.8120 - val_loss: 0.7944 - val_accuracy: 0.8182


In [12]:
best_hps.get('depth_of_network')

4